# MyUR3e Class Tutorial

## Initial Setup

In [ ]:
# First import and initiate rclpy (ROS2)
import rclpy
import numpy as np
rclpy.init()

# Then import MyUR3e and create an instance of the class
import MyUR3e
myrobot = MyUR3e.MyUR3e()

# You can also create instances of joint_states and tool_wrench if you would like (or access them through myrobot).
# These classes contain ROS2 nodes that subscribe to ROS2 topics and can help us access useful data.
joint_states = myrobot.joint_states
tool_wrench = myrobot.tool_wrench

# We also might want these libraries to help us out.
import time
from IPython.display import clear_output

# Lastly if you are having trouble with your code:
# Uncomment the code below for Debugging Logs
# myrobot.get_logger().set_level(rclpy.logging.LoggingSeverity.DEBUG)

## How to Use The Data Classes

Check out the loop below to see how to access data. 

For joint_states, the get_global() function will return the coordinates of the end effector relative to the the base of the robot in a six number array. (Can you guess what the last three numbers indicate?). The get_joints() function returns information about each joint and therefore is formatted differently. We can get information about the angle (position), velocity (angular velocity), and effort (torqe) of each joint.

The tool_wrench class gives us access to data from a 6-axis force sensor in the end effector.

See if you can fill in the empty values. If you want to see continuous data turn the for loop into a while loop.

In [ ]:
# Monitor MyUR3e Data

for i in range(10):
    clear_output(wait=True)
    for i in range(0,6):
        print(f"Joint States: {joint_states.get_joints()['name'][i]}")
        print(f"    Position: ??")
        print(f"    Velocity: ??")
        print(f"    Effort: {joint_states.get_joints()['effort'][i]}")
    print(f"Joint States Global:")
    print(f"    Position: {joint_states.get_global()}")
    print(f"Tool Wrench")
    print(f"    Force: {tool_wrench.get()['force']}")
    print(f"    Torque: ??")
    time.sleep(0.05)

## How to Move the Arm

The MyUR3e class has a function called move_global() that we will use to move the arm. This allows us to tell the arm where in 3D space we want the end effector to be. Inside of the MyUR3e class, we take these values and compute the angles of all six joints so that the end effector is exactly at that position. There is also a move_joints() function that gives you control over the angle of each joint, but using move_global is much easier! (If can code the end effector to move from in a line using the move_joints() function I will personally buy you a snack).

In [ ]:
# These two lines of code do the same thing:

# This one uses global coordinates (relative to the table)
myrobot.move_global([[0.01, 0.30, 0.23, 0, 0, 0]],time_step=5,sim=True)

# This one uses joint angles
'''
myrobot.move_joints([1.0861316919326782, 
                     -1.6310514211654663, 
                     1.9829953908920288, 
                     -1.9211959838867188, 
                     -1.5716094970703125, 
                     -0.48466402292251587], 
                    time_step=5)
'''

# The time_step parameter tells the arm how long it should take to get to that point.
# It is important that you don't make this variable very small otherwise the arm will move very fast!

## How to Create a Trajectory

We just learned how to move the arm to a single point. But what about if we want the arm to move across a whole path? The code below will move the end effector in a smooth tilted circle. Instead of passing the move_global() function an array of an array, we pass it an array of many arrays that each represent a sequential point in space.

Make sure to keep an eye on the time_step variable! We are only using a small time_step of 0.1 because we are moving many small distances. But if you used a time_step of 0.1 on a larger distance the arm would move way too fast!

In [ ]:
steps = 100 # fidelity of the circle
r = 0.05 # radius
c = [0.2,0.2,0.25] # center

circle = [] # array of points
for t in range(steps):
    x = c[0] + r * np.cos(t*2*np.pi/(steps))
    y = c[1] + r * np.sin(t*2 * np.pi / (steps))
    z = c[2] + x * 0.5
    circle.append([x,y,z,0.0,0.0,0.0])

# IMPORTANT TO READ !!!
# Here we represent time_step as a tuple (A,B) where A is the used for the first time_step and B for the rest.
# This is a safety feature so that we can specify a different time_step for the first coordinate than the rest.
# For example, if the robot arm was far away from the first coordinate in the trajectory, we would want to specify
# time_step A to be long so the arm moves slowly to its starting pose but time_step B to be short so that the
# trajectory doesn't take forever.
myrobot.move_global(circle,time_step=(2,0.1),sim=True)